<a href="https://colab.research.google.com/github/PaulSZH95/serve_ollama_as_restapi/blob/main/run_ollama_on_colab_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%writefile .env
# ngrok_token=add_your_auth_token_here

Writing .env


In [2]:
# %%writefile setup.sh
# #!/bin/bash

# # Load the environment variables from .env file
# source .env

# apt-get install lshw && \
# curl https://ollama.ai/install.sh | sh && \
# pip install aiohttp pyngrok python-dotenv && \
# ngrok config add-authtoken $ngrok_token

Writing something.sh


In [ ]:
# !chmod +x setup.sh
# !./setup.sh

In [4]:
!rm -r /content/sample_data/

replace mistral with the model you want



In [ ]:
%%shell

# Start "ollama serve" in the background using nohup
nohup ollama serve > /dev/null 2>&1 &
sleep 10
ollama pull mistral

In [6]:
import os
from pyngrok import conf
from dotenv import load_dotenv

load_dotenv()

ngrok_token = os.getenv("ngrok_token")
os.environ["ngrok"] = ngrok_token
conf.get_default().auth_token = ngrok_token

In [ ]:
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )
#     run_process(['ollama', 'pull','mistral']),
await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ngrok', 'http', '--log', 'stderr', '11434']),
)

Output of public url e.g. https://797e-34-74-220-15.ngrok-free.app/api/chat